### For the program augmented prompt, add "Here is the script that I will use to verify your output: "INSERT SCRIPT" to the end of the direct prompt.

# Time-Dependent Delivery Network

Design a time-dependent delivery network based on the specifications below.

## Problem Description:
Create a delivery network that schedules deliveries across multiple locations using a fleet of vehicles. The network must account for vehicle capacities, location storage capacities, delivery time windows, dynamic travel times, and vehicle speeds to ensure efficient and timely deliveries.

## Constraints:
1. **Locations:**
   - **Total Locations:** 15, labeled from `L0` to `L14`.
   - **Attributes:**
     - **Storage Capacity:** Each location has a storage capacity specified in kilograms (kg). Example: `L0` has a capacity of 500 kg.
     - **Time Window:** Each location has a delivery time window represented as a list of two integers `[start_hour, end_hour]` in 24-hour format. Example: `L3` has a time window of `[9, 11]` corresponding to 09:00-11:00.

2. **Vehicles:**
   - **Total Vehicles:** 7, labeled from `V1` to `V7`.
   - **Attributes:**
     - **Capacity:** Each vehicle has a specific capacity in kilograms (kg). Example: `V1` has a capacity of 100 kg.
     - **Speed:** Each vehicle has a defined speed in kilometers per hour (km/h). Example: `V1` travels at 60 km/h.

3. **Edges (Routes):**
   - **Definition:** Represents travel paths between two distinct locations.
   - **Attributes:**
     - **From:** The starting location ID (e.g., `L0`).
     - **To:** The destination location ID (e.g., `L1`).
     - **Base Travel Time:** The fundamental travel time for the route in minutes.
     - **Hourly Adjustments:** A dictionary where keys are time ranges in the format `"HH-HH"` (24-hour format) and values are additional travel time in minutes applicable during those hours. Example: `{"8-10": 15}` adds 15 minutes to the base travel time between 08:00-10:00.
     - **Maximum Weight Limit:** The maximum weight a vehicle can carry on that route in kilograms (kg).

4. **Operational Constraints:**
   - **Storage Capacity Compliance:** The sum of incoming goods to any location must not exceed its storage capacity.
   - **Vehicle Capacity Compliance:** No vehicle should exceed its capacity on any edge it traverses.
   - **Time Window Compliance:** Departures and arrivals must respect the time windows of locations. Specifically:
     - **Departure Time:** Must be within the `from` location's time window.
     - **Arrival Time:** Must be within the `to` location's time window.
     - **Loading Time:** Assume a fixed loading time of 10 minutes at each location, which must be accounted for when scheduling departures.

## Required Output Format:
Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
locations = [
    {"id": "L0", "capacity": int, "time_window": [int, int]},
    {"id": "L1", "capacity": int, "time_window": [int, int]},
    # ... all locations (L2-L14)
]

vehicles = [
    {"id": "V1", "capacity": int, "speed": int},
    {"id": "V2", "capacity": int, "speed": int},
    # ... all vehicles (V3-V7)
]

edges = [
    {"from": "L0", "to": "L1", "base_time": int, "hourly_adjust": {"HH-HH": int}, "max_weight": int},
    {"from": "L1", "to": "L2", "base_time": int, "hourly_adjust": {"HH-HH": int}, "max_weight": int},
    # ... more edges
]


In [65]:
from collections import defaultdict
import re

def verify_time_dependent_delivery_network(locations, vehicles, edges):
    verification_results = {
        "output_structure": {"passed": False, "message": ""},
        "storage_capacity_compliance": {"passed": False, "message": ""},
        "vehicle_capacity_compliance": {"passed": False, "message": ""},
        "time_window_compliance": {"passed": False, "message": ""},
        "overall_passed": False,
        "errors": []
    }
    def add_error(message):
        verification_results["errors"].append(message)

    if not isinstance(locations, list):
        add_error("'locations' should be a list.")
    elif len(locations) != 15:
        add_error(f"Number of locations is {len(locations)}; expected 15.")
    else:
        location_ids = set()
        location_valid = True
        for loc in locations:
            if not isinstance(loc, dict):
                add_error("Each location should be a dictionary.")
                location_valid = False
                continue
            required_loc_keys = {"id", "capacity", "time_window"}
            if not required_loc_keys.issubset(loc.keys()):
                add_error(f"Location {loc.get('id', 'Unknown')} is missing required keys: {required_loc_keys - set(loc.keys())}.")
                location_valid = False
                continue
            if not re.fullmatch(r"L\d+", loc["id"]):
                add_error(f"Location ID '{loc['id']}' is invalid. Expected format 'L0' to 'L14'.")
                location_valid = False
            location_ids.add(loc["id"])
            if not isinstance(loc["capacity"], int) or loc["capacity"] <= 0:
                add_error(f"Location '{loc['id']}' has invalid 'capacity'. Must be a positive integer.")
                location_valid = False
            if (not isinstance(loc["time_window"], list) or
                len(loc["time_window"]) != 2 or
                not all(isinstance(t, int) for t in loc["time_window"])):
                add_error(f"Location '{loc['id']}' has invalid 'time_window'. Must be a list of two integers.")
                location_valid = False
            else:
                start, end = loc["time_window"]
                if not (0 <= start < end <= 24):
                    add_error(f"Location '{loc['id']}' has invalid 'time_window' values: {loc['time_window']}. Must satisfy 0 <= start < end <= 24.")
                    location_valid = False
        if location_valid:
            verification_results["output_structure"]["passed"] = True
        else:
            verification_results["output_structure"]["message"] = "Errors found in 'locations' structure."
    
    if not isinstance(vehicles, list):
        add_error("'vehicles' should be a list.")
    elif len(vehicles) != 7:
        add_error(f"Number of vehicles is {len(vehicles)}; expected 7.")
    else:
        vehicle_ids = set()
        vehicle_valid = True
        vehicle_capacities = []
        for veh in vehicles:
            if not isinstance(veh, dict):
                add_error("Each vehicle should be a dictionary.")
                vehicle_valid = False
                continue
            required_veh_keys = {"id", "capacity", "speed"}
            if not required_veh_keys.issubset(veh.keys()):
                add_error(f"Vehicle {veh.get('id', 'Unknown')} is missing required keys: {required_veh_keys - set(veh.keys())}.")
                vehicle_valid = False
                continue
            if not re.fullmatch(r"V\d+", veh["id"]):
                add_error(f"Vehicle ID '{veh['id']}' is invalid. Expected format 'V1' to 'V7'.")
                vehicle_valid = False
            vehicle_ids.add(veh["id"])
            if not isinstance(veh["capacity"], int) or veh["capacity"] <= 0:
                add_error(f"Vehicle '{veh['id']}' has invalid 'capacity'. Must be a positive integer.")
                vehicle_valid = False
            else:
                vehicle_capacities.append(veh["capacity"])
            if not isinstance(veh["speed"], int) or veh["speed"] <= 0:
                add_error(f"Vehicle '{veh['id']}' has invalid 'speed'. Must be a positive integer.")
                vehicle_valid = False
        if vehicle_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'vehicles' structure."
    
    if not isinstance(edges, list):
        add_error("'edges' should be a list.")
    elif len(edges) == 0:
        add_error("No edges defined in the network.")
    else:
        edge_pairs = set()
        incoming_weights = defaultdict(int)
        edge_valid = True
        for edge in edges:
            if not isinstance(edge, dict):
                add_error("Each edge should be a dictionary.")
                edge_valid = False
                continue
            required_edge_keys = {"from", "to", "base_time", "hourly_adjust", "max_weight"}
            if not required_edge_keys.issubset(edge.keys()):
                add_error(f"Edge from '{edge.get('from', 'Unknown')}' to '{edge.get('to', 'Unknown')}' is missing required keys: {required_edge_keys - set(edge.keys())}.")
                edge_valid = False
                continue
            from_id = edge["from"]
            to_id = edge["to"]
            if from_id not in location_ids:
                add_error(f"Edge 'from' location '{from_id}' is not defined among locations.")
                edge_valid = False
            if to_id not in location_ids:
                add_error(f"Edge 'to' location '{to_id}' is not defined among locations.")
                edge_valid = False
            pair = (from_id, to_id)
            if pair in edge_pairs:
                add_error(f"Duplicate edge detected from '{from_id}' to '{to_id}'.")
                edge_valid = False
            else:
                edge_pairs.add(pair)
            if not isinstance(edge["base_time"], int) or edge["base_time"] <= 0:
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'base_time'. Must be a positive integer.")
                edge_valid = False
            if not isinstance(edge["hourly_adjust"], dict):
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'hourly_adjust'. Must be a dictionary.")
                edge_valid = False
            else:
                for time_range, adjustment in edge["hourly_adjust"].items():
                    if not re.fullmatch(r"\d{1,2}-\d{1,2}", time_range):
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid time range '{time_range}' in 'hourly_adjust'. Expected format 'HH-HH'.")
                        edge_valid = False
                        continue
                    start, end = map(int, time_range.split('-'))
                    if not (0 <= start < end <= 24):
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid time range values '{time_range}'. Must satisfy 0 <= start < end <= 24.")
                        edge_valid = False
                    if not isinstance(adjustment, int) or adjustment < 0:
                        add_error(f"Edge from '{from_id}' to '{to_id}' has invalid adjustment value '{adjustment}' for time range '{time_range}'. Must be a non-negative integer.")
                        edge_valid = False
            if not isinstance(edge["max_weight"], int) or edge["max_weight"] <= 0:
                add_error(f"Edge from '{from_id}' to '{to_id}' has invalid 'max_weight'. Must be a positive integer.")
                edge_valid = False
            else:
                incoming_weights[to_id] += edge["max_weight"]
        if edge_valid:
            verification_results["output_structure"]["passed"] = verification_results["output_structure"]["passed"] and True
        else:
            verification_results["output_structure"]["message"] += " Errors found in 'edges' structure."
    

    storage_compliance_passed = True
    storage_errors = []
    loc_capacity_map = {loc["id"]: loc["capacity"] for loc in locations}
    for loc_id, total_incoming in incoming_weights.items():
        capacity = loc_capacity_map.get(loc_id, 0)
        if total_incoming > capacity:
            storage_compliance_passed = False
            storage_errors.append(f"Total incoming 'max_weight' to location '{loc_id}' is {total_incoming} kg, exceeding its storage capacity of {capacity} kg.")
    if storage_compliance_passed:
        verification_results["storage_capacity_compliance"]["passed"] = True
    else:
        verification_results["storage_capacity_compliance"]["message"] = " ".join(storage_errors)
        add_error("Storage Capacity Compliance Check Failed.")
    
    vehicle_capacity_passed = True
    vehicle_errors = []
    if not vehicle_capacities:
        vehicle_capacity_passed = False
        vehicle_errors.append("No valid vehicle capacities found.")
    else:
        max_vehicle_capacity = max(vehicle_capacities)
        for edge in edges:
            if edge["max_weight"] < max_vehicle_capacity:
                vehicle_capacity_passed = False
                vehicle_errors.append(f"Edge from '{edge['from']}' to '{edge['to']}' has 'max_weight' {edge['max_weight']} kg, which is less than the maximum vehicle capacity of {max_vehicle_capacity} kg.")
    if vehicle_capacity_passed:
        verification_results["vehicle_capacity_compliance"]["passed"] = True
    else:
        verification_results["vehicle_capacity_compliance"]["message"] = " ".join(vehicle_errors)
        add_error("Vehicle Capacity Compliance Check Failed.")
    
    time_window_passed = True
    time_window_errors = []
    loc_time_map = {loc["id"]: loc["time_window"] for loc in locations}
    for edge in edges:
        from_id = edge["from"]
        to_id = edge["to"]
        base_time = edge["base_time"]  
        hourly_adjust = edge["hourly_adjust"] 
        max_adjust = max(hourly_adjust.values()) if hourly_adjust else 0
        total_travel_time = base_time + max_adjust  # in minutes
        total_travel_hours = (total_travel_time + 59) // 60 
        from_tw_start, from_tw_end = loc_time_map[from_id]
        to_tw_start, to_tw_end = loc_time_map[to_id]
        feasible = False
        for dep_hour in range(from_tw_start, from_tw_end):
            dep_time = dep_hour + 0.17 
            arr_time = dep_time + (total_travel_time / 60)
            if to_tw_start <= arr_time <= to_tw_end:
                feasible = True
                break
        if not feasible:
            time_window_passed = False
            time_window_errors.append(f"Edge from '{from_id}' to '{to_id}' with total travel time {total_travel_time} minutes does not fit within the time windows of '{from_id}' [{from_tw_start}-{from_tw_end}] and '{to_id}' [{to_tw_start}-{to_tw_end}].")
    if time_window_passed:
        verification_results["time_window_compliance"]["passed"] = True
    else:
        verification_results["time_window_compliance"]["message"] = " ".join(time_window_errors)
        add_error("Time Window Compliance Check Failed.")
    
    overall_passed = all([
        verification_results["output_structure"]["passed"],
        verification_results["storage_capacity_compliance"]["passed"],
        verification_results["vehicle_capacity_compliance"]["passed"],
        verification_results["time_window_compliance"]["passed"]
    ])
    verification_results["overall_passed"] = overall_passed
    
    return overall_passed, verification_results

locations = [  
    {"id": "L0", "capacity": 500, "time_window": [8, 18]},  
    {"id": "L1", "capacity": 350, "time_window": [9, 15]},  
    {"id": "L2", "capacity": 350, "time_window": [10, 16]},  
    {"id": "L3", "capacity": 400, "time_window": [9, 11]},  
    {"id": "L4", "capacity": 400, "time_window": [10, 14]},  
    {"id": "L5", "capacity": 350, "time_window": [8, 12]},  
    {"id": "L6", "capacity": 450, "time_window": [11, 17]},  
    {"id": "L7", "capacity": 380, "time_window": [8, 14]},  
    {"id": "L8", "capacity": 320, "time_window": [12, 18]},  
    {"id": "L9", "capacity": 280, "time_window": [9, 13]},  
    {"id": "L10", "capacity": 420, "time_window": [10, 15]},  
    {"id": "L11", "capacity": 370, "time_window": [11, 16]},  
    {"id": "L12", "capacity": 290, "time_window": [8, 13]},  
    {"id": "L13", "capacity": 330, "time_window": [9, 17]},  
    {"id": "L14", "capacity": 310, "time_window": [10, 18]}  
]  

vehicles = [  
    {"id": "V1", "capacity": 100, "speed": 60},  
    {"id": "V2", "capacity": 150, "speed": 55},  
    {"id": "V3", "capacity": 120, "speed": 65},  
    {"id": "V4", "capacity": 180, "speed": 50},  
    {"id": "V5", "capacity": 130, "speed": 58},  
    {"id": "V6", "capacity": 200, "speed": 45},  
    {"id": "V7", "capacity": 160, "speed": 52}  
]  

edges = [  
    {"from": "L0", "to": "L1", "base_time": 30, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L0", "to": "L2", "base_time": 45, "hourly_adjust": {"8-10": 20, "16-18": 25}, "max_weight": 200},  
    {"from": "L0", "to": "L3", "base_time": 40, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L0", "to": "L4", "base_time": 35, "hourly_adjust": {"8-10": 20, "16-18": 15}, "max_weight": 200},  
    {"from": "L1", "to": "L5", "base_time": 25, "hourly_adjust": {"8-10": 10, "16-18": 15}, "max_weight": 200},  
    {"from": "L1", "to": "L6", "base_time": 40, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L2", "to": "L7", "base_time": 30, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L2", "to": "L8", "base_time": 35, "hourly_adjust": {"8-10": 20, "16-18": 25}, "max_weight": 200},  
    {"from": "L3", "to": "L9", "base_time": 25, "hourly_adjust": {"8-10": 10, "16-18": 15}, "max_weight": 200},  
    {"from": "L3", "to": "L10", "base_time": 30, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L4", "to": "L11", "base_time": 40, "hourly_adjust": {"8-10": 20, "16-18": 25}, "max_weight": 200},  
    {"from": "L4", "to": "L12", "base_time": 35, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L5", "to": "L13", "base_time": 30, "hourly_adjust": {"8-10": 10, "16-18": 15}, "max_weight": 200},  
    {"from": "L6", "to": "L14", "base_time": 25, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L7", "to": "L0", "base_time": 50, "hourly_adjust": {"8-10": 20, "16-18": 25}, "max_weight": 200},  
    {"from": "L8", "to": "L1", "base_time": 45, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L9", "to": "L2", "base_time": 40, "hourly_adjust": {"8-10": 20, "16-18": 25}, "max_weight": 200},  
    {"from": "L10", "to": "L3", "base_time": 35, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L11", "to": "L4", "base_time": 30, "hourly_adjust": {"8-10": 10, "16-18": 15}, "max_weight": 200},  
    {"from": "L12", "to": "L5", "base_time": 40, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200},  
    {"from": "L13", "to": "L6", "base_time": 35, "hourly_adjust": {"8-10": 20, "16-18": 25}, "max_weight": 200},  
    {"from": "L14", "to": "L7", "base_time": 30, "hourly_adjust": {"8-10": 15, "16-18": 20}, "max_weight": 200}  
]  
verification_output = verify_time_dependent_delivery_network(locations, vehicles, edges)
print(verification_output)

(False, {'output_structure': {'passed': True, 'message': ''}, 'storage_capacity_compliance': {'passed': False, 'message': "Total incoming 'max_weight' to location 'L1' is 400 kg, exceeding its storage capacity of 350 kg. Total incoming 'max_weight' to location 'L2' is 400 kg, exceeding its storage capacity of 350 kg. Total incoming 'max_weight' to location 'L5' is 400 kg, exceeding its storage capacity of 350 kg. Total incoming 'max_weight' to location 'L7' is 400 kg, exceeding its storage capacity of 380 kg."}, 'vehicle_capacity_compliance': {'passed': True, 'message': ''}, 'time_window_compliance': {'passed': False, 'message': "Edge from 'L10' to 'L3' with total travel time 55 minutes does not fit within the time windows of 'L10' [10-15] and 'L3' [9-11]."}, 'overall_passed': False, 'errors': ['Storage Capacity Compliance Check Failed.', 'Time Window Compliance Check Failed.']})


In [66]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama32", 
    "GPT4o", "Claude35Sonnet", "Gemini20Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1", "Llama318B", "GrokV3","Claude37Sonnet"]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "timeDependentDeliveryNetwork"


def get_user_input():
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "locations": locations,
    "vehicles": vehicles,
    "edges": edges,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


Available Models:
1. GPT4omini
2. Claude35Haiku
3. Gemini20Flash
4. DeepSeekV3
5. Llama32
6. GPT4o
7. Claude35Sonnet
8. Gemini20Pro
9. Llama31405B
10. DeepSeekR1
11. o3mini
12. o1
13. Llama318B
14. GrokV3
15. Claude37Sonnet
Select a model by number: 15

Available Prompt Types:
1. directprompt
2. iterativefeedback
3. programaugmented
Select a prompt type by number: 3

Enter the run number (1-5): 5
Save is succesful! results/timeDependentDeliveryNetwork_Claude37Sonnet_programaugmented_run5_output.json


### # Directed Social Network with Influence Relationships

Design a directed social network based on the specifications below.

## Problem Description:
Create a social network graph representing influence relationships among users. Each user has specific attributes, and influence connections must adhere to defined constraints to maintain the integrity and intended structure of the network.

## Constraints:
1. **Users:**
   - Total of 20 users labeled from U0 to U19.
   - Each user has a "trust_score" ranging from 0 to 100.
   - Each user belongs to a "category" based on their trust score:
     - "celebrity" (trust_score ≥ 80)
     - "expert" (70 ≤ trust_score < 80)
     - "regular" (trust_score < 50)

2. **Edges (Influence Relationships):**
   - Directed edges where Ux → Uy indicates that Ux influences Uy.
   - **No self-loops:** A user cannot influence themselves.
   - **Category Constraints:**
     - **Celebrities:** Must have at least 5 outgoing edges.
     - **Regular Users:** Cannot influence experts.
   - **Graph Structure:**
     - The graph must be acyclic (no cycles in influence relationships).

### Required Output Format:
Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**


users = [
    {"id": "U0", "trust_score": int, "category": "celebrity"},
    {"id": "U1", "trust_score": int, "category": "regular"},
    {"id": "U2", "trust_score": int, "category": "expert"},
    # ... all users (U3-U19)
]

edges = [
    {"from": "U0", "to": "U1"},
    {"from: "U0", "to": "U2"},
    {"from": "U2", "to": "U3"},
    # ... more edges
]


In [144]:
from collections import defaultdict  
def verify_social_network(users, edges):
    # Initialize each constraint as a dict with passed and message.
    results = {
        "valid_no_self_loops": {"passed": True, "message": ""},
        "valid_celebrity_outgoing": {"passed": True, "message": ""},
        "valid_regular_to_expert": {"passed": True, "message": ""},
        "valid_acyclic": {"passed": True, "message": ""},
        "valid_user_attributes": {"passed": True, "message": ""},
        "valid_edge_structure": {"passed": True, "message": ""},
        "valid_total_users": {"passed": True, "message": ""},
        "valid_categories": {"passed": True, "message": ""},
        "valid_trust_scores": {"passed": True, "message": ""}
    }
    errors = []

    # Check users list and count
    if not isinstance(users, list):
        results["valid_total_users"] = {"passed": False, "message": "'users' should be a list."}
        errors.append("'users' should be a list.")
    elif len(users) != 20:
        results["valid_total_users"] = {"passed": False, "message": f"Number of users is {len(users)}; expected 20."}
        errors.append(f"Number of users is {len(users)}; expected 20.")

    expected_user_ids = {f"U{i}" for i in range(20)}
    actual_user_ids = set()
    user_map = {}
    for user in users:
        if not isinstance(user, dict):
            results["valid_user_attributes"] = {"passed": False, "message": "Each user should be a dictionary."}
            errors.append("Each user should be a dictionary.")
            continue
        required_keys = {"id", "trust_score", "category"}
        if not required_keys.issubset(user.keys()):
            missing = required_keys - set(user.keys())
            msg = f"User {user.get('id', 'Unknown')} is missing keys: {missing}."
            results["valid_user_attributes"]["passed"] = False
            results["valid_user_attributes"]["message"] += msg + " "
            errors.append(msg)
            continue
        user_id = user["id"]
        if user_id not in expected_user_ids:
            msg = f"User ID '{user_id}' is invalid. Expected IDs from U0 to U19."
            results["valid_user_attributes"]["passed"] = False
            results["valid_user_attributes"]["message"] += msg + " "
            errors.append(msg)
        actual_user_ids.add(user_id)
        user_map[user_id] = user
        trust_score = user["trust_score"]
        if not isinstance(trust_score, int) or not (0 <= trust_score <= 100):
            msg = f"User '{user_id}' has invalid 'trust_score': {trust_score}."
            results["valid_trust_scores"]["passed"] = False
            results["valid_trust_scores"]["message"] += msg + " "
            errors.append(msg)
        category = user["category"]
        if trust_score >= 80:
            if category != "celebrity":
                msg = f"User '{user_id}' has trust_score {trust_score} but category '{category}'; should be 'celebrity'."
                results["valid_categories"]["passed"] = False
                results["valid_categories"]["message"] += msg + " "
                errors.append(msg)
        elif 70 <= trust_score < 80:
            if category != "expert":
                msg = f"User '{user_id}' has trust_score {trust_score} but category '{category}'; should be 'expert'."
                results["valid_categories"]["passed"] = False
                results["valid_categories"]["message"] += msg + " "
                errors.append(msg)
        elif trust_score < 50:
            if category != "regular":
                msg = f"User '{user_id}' has trust_score {trust_score} but category '{category}'; should be 'regular'."
                results["valid_categories"]["passed"] = False
                results["valid_categories"]["message"] += msg + " "
                errors.append(msg)
        else:
            msg = f"User '{user_id}' has trust_score {trust_score} which is between 50 and 70; no valid category defined."
            results["valid_categories"]["passed"] = False
            results["valid_categories"]["message"] += msg + " "
            errors.append(msg)
    missing_users = expected_user_ids - actual_user_ids
    if missing_users:
        msg = f"Missing user IDs: {missing_users}."
        results["valid_total_users"]["passed"] = False
        results["valid_total_users"]["message"] += msg + " "
        errors.append(msg)
    
    # Check edges
    if not isinstance(edges, list):
        results["valid_edge_structure"] = {"passed": False, "message": "'edges' should be a list."}
        errors.append("'edges' should be a list.")
    else:
        adj = defaultdict(list)
        for edge in edges:
            if not isinstance(edge, dict):
                results["valid_edge_structure"] = {"passed": False, "message": "Each edge should be a dictionary."}
                errors.append("Each edge should be a dictionary.")
                continue
            required_edge_keys = {"from", "to"}
            if not required_edge_keys.issubset(edge.keys()):
                missing = required_edge_keys - set(edge.keys())
                msg = f"Edge is missing keys: {missing}."
                results["valid_edge_structure"]["passed"] = False
                results["valid_edge_structure"]["message"] += msg + " "
                errors.append(msg)
                continue
            from_id = edge["from"]
            to_id = edge["to"]
            if from_id not in expected_user_ids:
                msg = f"Edge 'from' user '{from_id}' is not valid."
                results["valid_edge_structure"]["passed"] = False
                results["valid_edge_structure"]["message"] += msg + " "
                errors.append(msg)
            if to_id not in expected_user_ids:
                msg = f"Edge 'to' user '{to_id}' is not valid."
                results["valid_edge_structure"]["passed"] = False
                results["valid_edge_structure"]["message"] += msg + " "
                errors.append(msg)
            adj[from_id].append(to_id)
    
    # Self-loop check
    no_self_loops_errors = []
    for edge in edges:
        if edge["from"] == edge["to"]:
            msg = f"Self-loop detected: {edge['from']} → {edge['to']}."
            no_self_loops_errors.append(msg)
    if no_self_loops_errors:
        results["valid_no_self_loops"]["passed"] = False
        results["valid_no_self_loops"]["message"] = " ".join(no_self_loops_errors)
        errors.extend(no_self_loops_errors)
    
    # Celebrity outgoing check: each celebrity must have at least 5 outgoing edges.
    celebrity_errors = []
    for user in users:
        if user["category"] == "celebrity":
            uid = user["id"]
            outgoing = len(adj[uid])
            if outgoing < 5:
                celebrity_errors.append(f"Celebrity '{uid}' has {outgoing} outgoing edges; expected at least 5.")
    if celebrity_errors:
        results["valid_celebrity_outgoing"]["passed"] = False
        results["valid_celebrity_outgoing"]["message"] = " ".join(celebrity_errors)
        errors.extend(celebrity_errors)
    
    # Regular-to-expert check: a regular user should not influence an expert.
    reg_to_expert_errors = []
    for edge in edges:
        from_user = user_map.get(edge["from"])
        to_user = user_map.get(edge["to"])
        if from_user and to_user:
            if from_user["category"] == "regular" and to_user["trust_score"] >= 70:
                reg_to_expert_errors.append(f"Regular user '{from_user['id']}' is influencing expert '{to_user['id']}'.")
    if reg_to_expert_errors:
        results["valid_regular_to_expert"]["passed"] = False
        results["valid_regular_to_expert"]["message"] = " ".join(reg_to_expert_errors)
        errors.extend(reg_to_expert_errors)
    
    # Acyclic check using DFS
    visited = set()
    rec_stack = set()
    def dfs(u):
        visited.add(u)
        rec_stack.add(u)
        for v in adj[u]:
            if v not in visited:
                if dfs(v):
                    return True
            elif v in rec_stack:
                return True
        rec_stack.remove(u)
        return False
    cycle_detected = False
    for uid in expected_user_ids:
        if uid not in visited:
            if dfs(uid):
                cycle_detected = True
                break
    if cycle_detected:
        msg = "Cycle detected in the network; the graph must be acyclic."
        results["valid_acyclic"]["passed"] = False
        results["valid_acyclic"]["message"] = msg
        errors.append(msg)
    
    overall_passed = all(results[k]["passed"] for k in results if k != "errors")
    return overall_passed, results


users = [
    {"id": "U0", "trust_score": 85, "category": "celebrity"},
    {"id": "U1", "trust_score": 90, "category": "celebrity"},
    {"id": "U2", "trust_score": 75, "category": "expert"},
    {"id": "U3", "trust_score": 72, "category": "expert"},
    {"id": "U4", "trust_score": 45, "category": "regular"},
    {"id": "U5", "trust_score": 40, "category": "regular"},
    {"id": "U6", "trust_score": 82, "category": "celebrity"},
    {"id": "U7", "trust_score": 78, "category": "expert"},
    {"id": "U8", "trust_score": 30, "category": "regular"},
    {"id": "U9", "trust_score": 35, "category": "regular"},
    {"id": "U10", "trust_score": 88, "category": "celebrity"},
    {"id": "U11", "trust_score": 70, "category": "expert"},
    {"id": "U12", "trust_score": 25, "category": "regular"},
    {"id": "U13", "trust_score": 20, "category": "regular"},
    {"id": "U14", "trust_score": 95, "category": "celebrity"},
    {"id": "U15", "trust_score": 73, "category": "expert"},
    {"id": "U16", "trust_score": 48, "category": "regular"},
    {"id": "U17", "trust_score": 15, "category": "regular"},
    {"id": "U18", "trust_score": 77, "category": "expert"},
    {"id": "U19", "trust_score": 42, "category": "regular"}
]

edges = [
    {"from": "U0", "to": "U2"},
    {"from": "U0", "to": "U3"},
    {"from": "U0", "to": "U4"},
    {"from": "U0", "to": "U5"},
    {"from": "U0", "to": "U7"},
    {"from": "U1", "to": "U2"},
    {"from": "U1", "to": "U3"},
    {"from": "U1", "to": "U8"},
    {"from": "U1", "to": "U9"},
    {"from": "U1", "to": "U11"},
    {"from": "U2", "to": "U4"},
    {"from": "U2", "to": "U5"},
    {"from": "U3", "to": "U8"},
    {"from": "U6", "to": "U7"},
    {"from": "U6", "to": "U9"},
    {"from": "U6", "to": "U11"},
    {"from": "U6", "to": "U12"},
    {"from": "U6", "to": "U13"},
    {"from": "U7", "to": "U16"},
    {"from": "U10", "to": "U11"},
    {"from": "U10", "to": "U12"},
    {"from": "U10", "to": "U13"},
    {"from": "U10", "to": "U15"},
    {"from": "U10", "to": "U16"},
    {"from": "U11", "to": "U17"},
    {"from": "U14", "to": "U15"},
    {"from": "U14", "to": "U16"},
    {"from": "U14", "to": "U17"},
    {"from": "U14", "to": "U18"},
    {"from": "U14", "to": "U19"},
    {"from": "U15", "to": "U19"},
    {"from": "U18", "to": "U12"}
]

verification_output = verify_social_network(users, edges)
print("Verification Output:", verification_output)



Verification Output: (True, {'valid_no_self_loops': {'passed': True, 'message': ''}, 'valid_celebrity_outgoing': {'passed': True, 'message': ''}, 'valid_regular_to_expert': {'passed': True, 'message': ''}, 'valid_acyclic': {'passed': True, 'message': ''}, 'valid_user_attributes': {'passed': True, 'message': ''}, 'valid_edge_structure': {'passed': True, 'message': ''}, 'valid_total_users': {'passed': True, 'message': ''}, 'valid_categories': {'passed': True, 'message': ''}, 'valid_trust_scores': {'passed': True, 'message': ''}})


In [145]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama32", 
    "GPT4o", "Claude35Sonnet", "Gemini20Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1", "Llama318B", "GrokV3","Claude37Sonnet"]


prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "directedSocialNetwork"


def get_user_input():
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "users":users,
    "edges":edges,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


Available Models:
1. GPT4omini
2. Claude35Haiku
3. Gemini20Flash
4. DeepSeekV3
5. Llama32
6. GPT4o
7. Claude35Sonnet
8. Gemini20Pro
9. Llama31405B
10. DeepSeekR1
11. o3mini
12. o1
13. Llama318B
14. GrokV3
15. Claude37Sonnet
Select a model by number: 14

Available Prompt Types:
1. directprompt
2. iterativefeedback
3. programaugmented
Select a prompt type by number: 3

Enter the run number (1-5): 5
Save is succesful! results/directedSocialNetwork_GrokV3_programaugmented_run5_output.json


# Quantum Circuit Design  

## Problem Description:
Design a quantum circuit consisting of multiple qubits and quantum gates. The circuit must adhere to specific constraints to ensure proper gate operations, circuit efficiency, and overall functionality. The design should incorporate structural elements like depth and a Directed Acyclic Graph (DAG) while simplifying some of the gate-related rules to enhance accessibility.

## Constraints:
1. **Qubits:**
   - **Total Qubits:** 10, labeled from `Q0` to `Q9`.
   - **Initialization:** All qubits must start in the |0⟩ state.

2. **Gates:**
   - **Types of Gates to Include:**
     - **Single-Qubit Gates:** Hadamard (H), Pauli-X (X), Pauli-Z (Z)
     - **Multi-Qubit Gates:** Controlled NOT (CNOT), SWAP
     - **Measurement:** Measure (Measure)
   - **Gate Operations:**
     - Each gate operates on specific qubits at designated times.
     - **CNOT Gates:** Must operate on qubits that are not adjacent (e.g., Q0 and Q2 are valid; Q0 and Q1 are invalid).
     - **SWAP Gates:** Must operate between pairs of qubits that have identical gate sequences up to that point.
     - **Measurements:** Each qubit can be measured only once and must be the last operation on that qubit.
   - **Gate Restrictions:**
     - **Gate Frequency:** No single-qubit gate can be applied more than twice consecutively on the same qubit.

3. **Circuit Structure:**
   - The circuit must be a Directed Acyclic Graph (DAG); no repeated times for the same qubit.
   - **Layered Operations:** Gates at the same time step must operate on disjoint sets of qubits (i.e., no two gates at the same time can act on the same qubit).
   - **Depth Constraint:** The total number of time steps (layers) must not exceed 30.

4. **Operational Constraints:**
   - **Circuit Reversibility:** Measurements must be the final operations on their respective qubits to maintain circuit reversibility.
   - **Gate Optimization:** The circuit should minimize the total number of gates while satisfying all other constraints.
   - **Final State:** After all operations, all qubits must either be measured or returned to the |0⟩ state.

## Required Output Format:

Provide the circuit details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
circuit = [
    {"gate": "H", "qubits": ["Q0"], "time": 1},
    {"gate": "CNOT", "qubits": ["Q0", "Q2"], "time": 2},
    {"gate": "X", "qubits": ["Q2"], "time": 3},
    {"gate": "Measure", "qubits": ["Q0"], "time": 4},
    # ... more gates
]


In [208]:
import re
from collections import defaultdict

import re
from collections import defaultdict

def verify_quantum_circuit(circuit):
    results = {
        "valid_output_structure": {"passed": True, "message": ""},
        "valid_gate_types": {"passed": True, "message": ""},
        "valid_qubits": {"passed": True, "message": ""},
        "valid_dag": {"passed": True, "message": ""},
        "valid_gate_precedences": {"passed": True, "message": ""},
        "valid_cnot_adjacency": {"passed": True, "message": ""},
        "valid_swap_constraints": {"passed": True, "message": ""},
        "valid_measurements": {"passed": True, "message": ""},
        "valid_gate_restrictions": {"passed": True, "message": ""},
        "layered_operations": {"passed": True, "message": ""},
        "depth_constraint": {"passed": True, "message": ""},
        "gate_optimization": {"passed": True, "message": ""},
        "final_state_compliance": {"passed": True, "message": ""}
    }
    errors = []
    valid_qubits = {f"Q{i}" for i in range(10)}
    single_qubit_gates = {"H", "X", "Z"}
    multi_qubit_gates = {"CNOT", "SWAP"}
    measurement_gate = "Measure"
    allowed_gates = single_qubit_gates.union(multi_qubit_gates).union({measurement_gate})
    
    qubit_operations = defaultdict(list)
    gate_sequences = defaultdict(list)
    time_layers = defaultdict(set)
    max_time_step = 0

    def add_error(key, message):
        results[key]["passed"] = False
        if results[key]["message"]:
            results[key]["message"] += " " + message
        else:
            results[key]["message"] = message
        errors.append(message)

    if not isinstance(circuit, list):
        add_error("valid_output_structure", "'circuit' should be a list.")
        return False, results

    for idx, gate_op in enumerate(circuit):
        if not isinstance(gate_op, dict):
            add_error("valid_output_structure", f"Gate operation at index {idx} is not a dictionary.")
            continue
        required_keys = {"gate", "qubits", "time"}
        if not required_keys.issubset(gate_op.keys()):
            missing = required_keys - set(gate_op.keys())
            add_error("valid_output_structure", f"Gate operation at index {idx} is missing keys: {missing}.")
            continue
        gate = gate_op["gate"]
        qubits = gate_op["qubits"]
        time = gate_op["time"]
        if gate not in allowed_gates:
            add_error("valid_gate_types", f"Invalid gate type '{gate}' at index {idx}. Allowed: {allowed_gates}.")
        if not isinstance(qubits, list) or not all(isinstance(q, str) for q in qubits):
            add_error("valid_output_structure", f"'qubits' for gate at index {idx} must be a list of strings.")
            continue
        for q in qubits:
            if q not in valid_qubits:
                add_error("valid_qubits", f"Invalid qubit ID '{q}' at index {idx}. Expected Q0 to Q9.")
        if not isinstance(time, int) or time < 0:
            add_error("valid_output_structure", f"Invalid 'time' value '{time}' at index {idx}. Must be a non-negative integer.")
            continue
        if time > max_time_step:
            max_time_step = time
        for q in qubits:
            qubit_operations[q].append((time, gate))
            gate_sequences[q].append(gate)
        for q in qubits:
            if q in time_layers[time]:
                add_error("layered_operations", f"Multiple gates operate on qubit '{q}' at time {time}.")
            time_layers[time].add(q)
        if gate == "CNOT":
            if len(qubits) != 2:
                add_error("valid_output_structure", f"CNOT gate at index {idx} must operate on exactly two qubits.")
        elif gate == "SWAP":
            if len(qubits) != 2:
                add_error("valid_output_structure", f"SWAP gate at index {idx} must operate on exactly two qubits.")
        elif gate in single_qubit_gates and gate != "Measure":
            if len(qubits) != 1:
                add_error("valid_output_structure", f"{gate} gate at index {idx} must operate on exactly one qubit.")
        elif gate == "Measure":
            if len(qubits) != 1:
                add_error("valid_output_structure", f"Measure gate at index {idx} must operate on exactly one qubit.")

    for q, ops in qubit_operations.items():
        times = [t for (t, _) in ops]
        if len(times) != len(set(times)):
            add_error("valid_dag", f"Qubit '{q}' has overlapping operations at the same time.")

    for q, gates in gate_sequences.items():
        previous = None
        count = 0
        for idx, g in enumerate(gates):
            if g in single_qubit_gates:
                if g == previous:
                    count += 1
                    if count > 2:
                        add_error("valid_gate_restrictions", f"Qubit '{q}' has more than two consecutive '{g}' gates at positions {idx-1} and {idx}.")
                else:
                    count = 1
            else:
                count = 0
            previous = g

    for idx, gate_op in enumerate(circuit):
        if gate_op["gate"] == "CNOT":
            q1, q2 = gate_op["qubits"]
            if abs(int(q1[1:]) - int(q2[1:])) < 2:
                add_error("valid_cnot_adjacency", f"CNOT gate at index {idx} operates on adjacent qubits '{q1}' and '{q2}'.")
    for idx, gate_op in enumerate(circuit):
        if gate_op["gate"] == "SWAP":
            q1, q2 = gate_op["qubits"]
            # For SWAP, check that prior gate sequences (excluding current SWAP) are identical.
            seq1 = gate_sequences[q1][:gate_sequences[q1].index("SWAP")]
            seq2 = gate_sequences[q2][:gate_sequences[q2].index("SWAP")]
            if seq1 != seq2:
                add_error("valid_swap_constraints", f"SWAP gate at index {idx} operates on qubits '{q1}' and '{q2}' with differing prior gate sequences.")
    measurement_counts = defaultdict(int)
    for gate_op in circuit:
        if gate_op["gate"] == "Measure":
            measurement_counts[gate_op["qubits"][0]] += 1
    for q, count in measurement_counts.items():
        if count > 1:
            add_error("valid_measurements", f"Qubit '{q}' is measured {count} times; only one measurement allowed.")
    if max_time_step > 30:
        add_error("depth_constraint", f"Circuit depth is {max_time_step} time steps; maximum allowed is 30.")
    for q, gates in gate_sequences.items():
        for i in range(1, len(gates)):
            if gates[i] == gates[i-1] and gates[i] in single_qubit_gates:
                add_error("gate_optimization", f"Redundant consecutive '{gates[i]}' gates on qubit '{q}' at positions {i-1} and {i}.")
    for q, gates in gate_sequences.items():
        if gates and gates[-1] != "Measure":
            add_error("final_state_compliance", f"Qubit '{q}' does not end with a Measurement (or resetting gate).")
    
    overall_passed = all(results[key]["passed"] for key in results)
    results["overall_passed"] = overall_passed
    return overall_passed, results



circuit = [
    # Layer 1 - Initial Hadamard gates to create superposition
    {"gate": "H", "qubits": ["Q0"], "time": 1},
    {"gate": "H", "qubits": ["Q2"], "time": 1},
    {"gate": "H", "qubits": ["Q4"], "time": 1},
    {"gate": "H", "qubits": ["Q6"], "time": 1},

    # Layer 2 - First set of CNOT gates (non-adjacent qubits)
    {"gate": "CNOT", "qubits": ["Q0", "Q2"], "time": 2},
    {"gate": "CNOT", "qubits": ["Q4", "Q6"], "time": 2},
    {"gate": "X", "qubits": ["Q8"], "time": 2},

    # Layer 3 - Pauli gates
    {"gate": "Z", "qubits": ["Q1"], "time": 3},
    {"gate": "X", "qubits": ["Q3"], "time": 3},
    {"gate": "Z", "qubits": ["Q5"], "time": 3},

    # Layer 4 - SWAP gates (Q1 and Q3 have identical sequences up to this point: [Z] and [X])
    {"gate": "SWAP", "qubits": ["Q2", "Q4"], "time": 4},
    {"gate": "CNOT", "qubits": ["Q6", "Q8"], "time": 4},

    # Layer 5 - Second set of single-qubit gates
    {"gate": "H", "qubits": ["Q7"], "time": 5},
    {"gate": "X", "qubits": ["Q9"], "time": 5},

    # Layers 6-15 - Measurements for all qubits
    {"gate": "Measure", "qubits": ["Q0"], "time": 6},
    {"gate": "Measure", "qubits": ["Q1"], "time": 7},
    {"gate": "Measure", "qubits": ["Q2"], "time": 8},
    {"gate": "Measure", "qubits": ["Q3"], "time": 9},
    {"gate": "Measure", "qubits": ["Q4"], "time": 10},
    {"gate": "Measure", "qubits": ["Q5"], "time": 11},
    {"gate": "Measure", "qubits": ["Q6"], "time": 12},
    {"gate": "Measure", "qubits": ["Q7"], "time": 13},
    {"gate": "Measure", "qubits": ["Q8"], "time": 14},
    {"gate": "Measure", "qubits": ["Q9"], "time": 15}
]

# Execute verification
verification_output = verify_quantum_circuit(circuit)
print("Verification Output:", verification_output)




Verification Output: (True, {'valid_output_structure': {'passed': True, 'message': ''}, 'valid_gate_types': {'passed': True, 'message': ''}, 'valid_qubits': {'passed': True, 'message': ''}, 'valid_dag': {'passed': True, 'message': ''}, 'valid_gate_precedences': {'passed': True, 'message': ''}, 'valid_cnot_adjacency': {'passed': True, 'message': ''}, 'valid_swap_constraints': {'passed': True, 'message': ''}, 'valid_measurements': {'passed': True, 'message': ''}, 'valid_gate_restrictions': {'passed': True, 'message': ''}, 'layered_operations': {'passed': True, 'message': ''}, 'depth_constraint': {'passed': True, 'message': ''}, 'gate_optimization': {'passed': True, 'message': ''}, 'final_state_compliance': {'passed': True, 'message': ''}, 'overall_passed': True})


In [209]:
import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama32", 
    "GPT4o", "Claude35Sonnet", "Gemini20Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1", "Llama318B", "GrokV3","Claude37Sonnet"]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "quantum"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "circuit": circuit,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


Available Models:
1. GPT4omini
2. Claude35Haiku
3. Gemini20Flash
4. DeepSeekV3
5. Llama32
6. GPT4o
7. Claude35Sonnet
8. Gemini20Pro
9. Llama31405B
10. DeepSeekR1
11. o3mini
12. o1
13. Llama318B
14. GrokV3
15. Claude37Sonnet
Select a model by number: 14

Available Prompt Types:
1. directprompt
2. iterativefeedback
3. programaugmented
Select a prompt type by number: 3

Enter the run number (1-5): 5
Save is succesful! results/quantum_GrokV3_programaugmented_run5_output.json


# Gene-Disease Association Network

Design a gene-disease association network based on the specifications below.

## Problem Description:
Create a bipartite network that models the associations between genes and diseases. This network will represent which genes are associated with which diseases, capturing the strength of each association. The network should adhere to defined constraints to ensure biological relevance and structural integrity.

## Constraints:
1. **Nodes:**
   - **Genes:**
     - Total of 20 genes labeled from G0 to G19.
     - Each gene has a "name" and a "function".
   - **Diseases:**
     - Total of 20 diseases labeled from D0 to D19.
     - Each disease has a "name" and a "severity_level" (e.g., "Low", "Medium", "High").
   
2. **Edges (Associations):**
   - Represents the association between a gene and a disease.
   - **Bipartite Constraint:** Associations can only exist between genes and diseases, not within the same set.
   - **Association Strength:** Each association has a "strength" value ranging from 0.0 to 1.0, indicating the confidence of the association.
   
3. **Degree Constraints:**
   - **Genes:**
     - Each gene must be associated with at least 2 and at most 5 diseases.
   - **Diseases:**
     - Each disease must be associated with at least 3 and at most 10 genes.
   
4. **Structural Constraints:**
   - The network must be bipartite; no edges should connect nodes within the same set (i.e., no gene-gene or disease-disease associations).
   - There should be no duplicate edges (i.e., each gene-disease pair is unique).

## Required Output Format:

Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
genes = [
    {"id": "G0", "name": "BRCA1", "function": "DNA repair"},
    {"id": "G1", "name": "TP53", "function": "Tumor suppression"},
    # ... all genes (G2-G19)
]

diseases = [
    {"id": "D0", "name": "Breast Cancer", "severity_level": "High"},
    {"id": "D1", "name": "Lung Cancer", "severity_level": "High"},
    # ... all diseases (D2-D19)
]

associations = [
    {"from": "G0", "to": "D0", "strength": 0.85},
    {"from": "G0", "to": "D2", "strength": 0.60},
    {"from": "G1", "to": "D0", "strength": 0.90},
    # ... more associations
]


In [266]:
import os
import json
import glob
import re
from collections import defaultdict

def verify_gene_disease_association_network(genes, diseases, associations):
    # Prepare the verification result container
    verification_results = {
        "defined_counts": {"passed": False, "message": ""},
        "valid_associations": {"passed": False, "message": ""},
        "degree_constraints": {"passed": False, "message": ""},
        "duplicate_associations": {"passed": False, "message": ""},
        "bipartite_constraint": {"passed": False, "message": ""}
    }
    error_messages = []

    # Check counts of nodes
    if len(genes) == 20 and len(diseases) == 20:
        verification_results["defined_counts"]["passed"] = True
    else:
        missing = []
        if len(genes) != 20:
            missing.append(f"genes count is {len(genes)}; expected 20.")
        if len(diseases) != 20:
            missing.append(f"diseases count is {len(diseases)}; expected 20.")
        verification_results["defined_counts"]["message"] = " ".join(missing)
        error_messages.append("Defined Counts Check Failed.")

    # Build maps for easy lookup
    gene_map = {gene["id"]: gene for gene in genes}
    disease_map = {disease["id"]: disease for disease in diseases}

    # Check associations:
    invalid_associations = []
    bipartite_errors = []
    for assoc in associations:
        from_id = assoc.get("from")
        to_id = assoc.get("to")
        strength = assoc.get("strength")

        # Must be from a gene and to a disease.
        if from_id not in gene_map:
            invalid_associations.append(f"Association from undefined gene '{from_id}' to '{to_id}'.")
        if to_id not in disease_map:
            invalid_associations.append(f"Association from '{from_id}' to undefined disease '{to_id}'.")

        # Bipartite check: if both endpoints are genes or both are diseases.
        if from_id in gene_map and to_id in gene_map:
            bipartite_errors.append(f"Association between gene '{from_id}' and gene '{to_id}' violates bipartite constraint.")
        if from_id in disease_map and to_id in disease_map:
            bipartite_errors.append(f"Association between disease '{from_id}' and disease '{to_id}' violates bipartite constraint.")

        # Check association strength
        if not isinstance(strength, (int, float)) or not (0.0 <= strength <= 1.0):
            invalid_associations.append(
                f"Association strength {strength} for '{from_id}' -> '{to_id}' is out of bounds (0.0 - 1.0)."
            )
    
    if not invalid_associations:
        verification_results["valid_associations"]["passed"] = True
    else:
        verification_results["valid_associations"]["message"] = " ".join(invalid_associations)
        error_messages.append("Valid Associations Check Failed.")

    if not bipartite_errors:
        verification_results["bipartite_constraint"]["passed"] = True
    else:
        verification_results["bipartite_constraint"]["message"] = " ".join(bipartite_errors)
        error_messages.append("Bipartite Constraint Check Failed.")

    # Degree constraints
    gene_degrees = defaultdict(int)
    disease_degrees = defaultdict(int)
    for assoc in associations:
        gene_id = assoc.get("from")
        disease_id = assoc.get("to")
        gene_degrees[gene_id] += 1
        disease_degrees[disease_id] += 1

    degree_errors = []
    for gene in genes:
        gene_id = gene["id"]
        degree = gene_degrees.get(gene_id, 0)
        if degree < 2 or degree > 5:
            degree_errors.append(f"Gene '{gene_id}' has {degree} associations; expected between 2 and 5.")

    for disease in diseases:
        disease_id = disease["id"]
        degree = disease_degrees.get(disease_id, 0)
        if degree < 3 or degree > 10:
            degree_errors.append(f"Disease '{disease_id}' has {degree} associations; expected between 3 and 10.")

    if not degree_errors:
        verification_results["degree_constraints"]["passed"] = True
    else:
        verification_results["degree_constraints"]["message"] = " ".join(degree_errors)
        error_messages.append("Degree Constraints Check Failed.")

    # Duplicate associations check
    seen_pairs = set()
    duplicate_errors = []
    for assoc in associations:
        pair = (assoc.get("from"), assoc.get("to"))
        if pair in seen_pairs:
            duplicate_errors.append(f"Duplicate association found between '{pair[0]}' and '{pair[1]}'.")
        else:
            seen_pairs.add(pair)
    
    if not duplicate_errors:
        verification_results["duplicate_associations"]["passed"] = True
    else:
        verification_results["duplicate_associations"]["message"] = " ".join(duplicate_errors)
        error_messages.append("Duplicate Associations Check Failed.")

    final_results = {
        "constraints": verification_results,
        "overall_passed": not error_messages,
        "errors": error_messages
    }
    return (not error_messages, final_results)

genes = [  
    {"id": "G0", "name": "BRCA1", "function": "DNA repair and tumor suppression"},  
    {"id": "G1", "name": "TP53", "function": "Tumor suppression and cell cycle regulation"},  
    {"id": "G2", "name": "EGFR", "function": "Cell growth and division signaling"},  
    {"id": "G3", "name": "KRAS", "function": "GTPase signaling in cell growth"},  
    {"id": "G4", "name": "PTEN", "function": "Tumor suppression and cell cycle regulation"},  
    {"id": "G5", "name": "APP", "function": "Neural growth and repair"},  
    {"id": "G6", "name": "CFTR", "function": "Chloride channel regulation"},  
    {"id": "G7", "name": "HBB", "function": "Oxygen transport in blood"},  
    {"id": "G8", "name": "APOE", "function": "Lipid metabolism and transport"},  
    {"id": "G9", "name": "TNF", "function": "Inflammatory response regulation"},  
    {"id": "G10", "name": "IL6", "function": "Immune response signaling"},  
    {"id": "G11", "name": "ACE2", "function": "Blood pressure regulation"},  
    {"id": "G12", "name": "MTHFR", "function": "Folate metabolism"},  
    {"id": "G13", "name": "COMT", "function": "Neurotransmitter metabolism"},  
    {"id": "G14", "name": "HTT", "function": "Neural development"},  
    {"id": "G15", "name": "SOD1", "function": "Antioxidant defense"},  
    {"id": "G16", "name": "DMD", "function": "Muscle cell structure and function"},  
    {"id": "G17", "name": "FMR1", "function": "RNA binding and neural development"},  
    {"id": "G18", "name": "LDLR", "function": "Cholesterol metabolism"},  
    {"id": "G19", "name": "NF1", "function": "Cell growth regulation"}  
]  

diseases = [  
    {"id": "D0", "name": "Breast Cancer", "severity_level": "High"},  
    {"id": "D1", "name": "Lung Cancer", "severity_level": "High"},  
    {"id": "D2", "name": "Alzheimer's Disease", "severity_level": "High"},  
    {"id": "D3", "name": "Cystic Fibrosis", "severity_level": "High"},  
    {"id": "D4", "name": "Huntington's Disease", "severity_level": "High"},  
    {"id": "D5", "name": "Type 2 Diabetes", "severity_level": "Medium"},  
    {"id": "D6", "name": "Coronary Heart Disease", "severity_level": "High"},  
    {"id": "D7", "name": "Asthma", "severity_level": "Medium"},  
    {"id": "D8", "name": "Parkinson's Disease", "severity_level": "High"},  
    {"id": "D9", "name": "Multiple Sclerosis", "severity_level": "High"},  
    {"id": "D10", "name": "Rheumatoid Arthritis", "severity_level": "Medium"},  
    {"id": "D11", "name": "Sickle Cell Anemia", "severity_level": "High"},  
    {"id": "D12", "name": "Hypertension", "severity_level": "Medium"},  
    {"id": "D13", "name": "Depression", "severity_level": "Medium"},  
    {"id": "D14", "name": "Schizophrenia", "severity_level": "High"},  
    {"id": "D15", "name": "Osteoporosis", "severity_level": "Medium"},  
    {"id": "D16", "name": "Muscular Dystrophy", "severity_level": "High"},  
    {"id": "D17", "name": "Celiac Disease", "severity_level": "Medium"},  
    {"id": "D18", "name": "Fragile X Syndrome", "severity_level": "High"},  
    {"id": "D19", "name": "Hypercholesterolemia", "severity_level": "Low"}  
]  

associations = [  
    # BRCA1 associations (G0)  
    {"from": "G0", "to": "D0", "strength": 0.95},  
    {"from": "G0", "to": "D1", "strength": 0.45},  
    {"from": "G0", "to": "D6", "strength": 0.35},  
    {"from": "G0", "to": "D3", "strength": 0.25},  
    {"from": "G0", "to": "D18", "strength": 0.30},  
    
    # TP53 associations (G1)  
    {"from": "G1", "to": "D0", "strength": 0.85},  
    {"from": "G1", "to": "D1", "strength": 0.90},  
    {"from": "G1", "to": "D8", "strength": 0.40},  
    {"from": "G1", "to": "D14", "strength": 0.30},  
    {"from": "G1", "to": "D16", "strength": 0.40},  
    
    # EGFR associations (G2)  
    {"from": "G2", "to": "D1", "strength": 0.90},  
    {"from": "G2", "to": "D0", "strength": 0.65},  
    {"from": "G2", "to": "D7", "strength": 0.35},  
    {"from": "G2", "to": "D10", "strength": 0.45},  
    {"from": "G2", "to": "D12", "strength": 0.25},  
    
    # KRAS associations (G3)  
    {"from": "G3", "to": "D1", "strength": 0.85},  
    {"from": "G3", "to": "D6", "strength": 0.40},  
    {"from": "G3", "to": "D5", "strength": 0.35},  
    {"from": "G3", "to": "D11", "strength": 0.30},  
    {"from": "G3", "to": "D18", "strength": 0.35},  
    
    # PTEN associations (G4)  
    {"from": "G4", "to": "D0", "strength": 0.80},  
    {"from": "G4", "to": "D9", "strength": 0.45},  
    {"from": "G4", "to": "D13", "strength": 0.30},  
    {"from": "G4", "to": "D14", "strength": 0.55},  
    {"from": "G4", "to": "D17", "strength": 0.30},  
    
    # APP associations (G5)  
    {"from": "G5", "to": "D2", "strength": 0.95},  
    {"from": "G5", "to": "D8", "strength": 0.50},  
    {"from": "G5", "to": "D13", "strength": 0.40},  
    {"from": "G5", "to": "D4", "strength": 0.55},  
    
    # CFTR associations (G6)  
    {"from": "G6", "to": "D3", "strength": 0.98},  
    {"from": "G6", "to": "D7", "strength": 0.35},  
    {"from": "G6", "to": "D17", "strength": 0.25},  
    
    # HBB associations (G7)  
    {"from": "G7", "to": "D11", "strength": 0.95},  
    {"from": "G7", "to": "D6", "strength": 0.40},  
    {"from": "G7", "to": "D12", "strength": 0.45},  
    {"from": "G7", "to": "D16", "strength": 0.35},  
    
    # APOE associations (G8)  
    {"from": "G8", "to": "D2", "strength": 0.85},  
    {"from": "G8", "to": "D6", "strength": 0.75},  
    {"from": "G8", "to": "D19", "strength": 0.80},  
    {"from": "G8", "to": "D5", "strength": 0.45},  
    {"from": "G8", "to": "D18", "strength": 0.40},  
    
    # TNF associations (G9)  
    {"from": "G9", "to": "D10", "strength": 0.85},  
    {"from": "G9", "to": "D7", "strength": 0.65},  
    {"from": "G9", "to": "D9", "strength": 0.70},  
    {"from": "G9", "to": "D17", "strength": 0.50},  
    {"from": "G9", "to": "D15", "strength": 0.35},  
    
    # IL6 associations (G10)  
    {"from": "G10", "to": "D10", "strength": 0.80},  
    {"from": "G10", "to": "D9", "strength": 0.65},  
    {"from": "G10", "to": "D5", "strength": 0.55},  
    {"from": "G10", "to": "D12", "strength": 0.40},  
    {"from": "G10", "to": "D11", "strength": 0.25},  
    
    # ACE2 associations (G11)  
    {"from": "G11", "to": "D12", "strength": 0.80},  
    {"from": "G11", "to": "D5", "strength": 0.50},  
    {"from": "G11", "to": "D6", "strength": 0.65},  
    {"from": "G11", "to": "D3", "strength": 0.20},  
    
    # MTHFR associations (G12)  
    {"from": "G12", "to": "D6", "strength": 0.60},  
    {"from": "G12", "to": "D13", "strength": 0.45},  
    {"from": "G12", "to": "D15", "strength": 0.40},  
    {"from": "G12", "to": "D19", "strength": 0.35},  
    {"from": "G12", "to": "D17", "strength": 0.40},  
    
    # COMT associations (G13)  
    {"from": "G13", "to": "D13", "strength": 0.70},  
    {"from": "G13", "to": "D14", "strength": 0.65},  
    {"from": "G13", "to": "D8", "strength": 0.45},  
    {"from": "G13", "to": "D4", "strength": 0.40},  
    
    # HTT associations (G14)  
    {"from": "G14", "to": "D4", "strength": 0.98},  
    {"from": "G14", "to": "D8", "strength": 0.40},  
    {"from": "G14", "to": "D13", "strength": 0.35},  
    {"from": "G14", "to": "D17", "strength": 0.35},  
    
    # SOD1 associations (G15)  
    {"from": "G15", "to": "D8", "strength": 0.65},  
    {"from": "G15", "to": "D9", "strength": 0.55},  
    {"from": "G15", "to": "D15", "strength": 0.40},  
    {"from": "G15", "to": "D2", "strength": 0.45},  
    {"from": "G15", "to": "D3", "strength": 0.30},  
    
    # DMD associations (G16)  
    {"from": "G16", "to": "D16", "strength": 0.95},  
    {"from": "G16", "to": "D6", "strength": 0.35},  
    {"from": "G16", "to": "D15", "strength": 0.30},  
    
    # FMR1 associations (G17)  
    {"from": "G17", "to": "D18", "strength": 0.95},  
    {"from": "G17", "to": "D13", "strength": 0.50},  
    {"from": "G17", "to": "D14", "strength": 0.45},  
    
    # LDLR associations (G18)  
    {"from": "G18", "to": "D19", "strength": 0.95},  
    {"from": "G18", "to": "D6", "strength": 0.75},  
    {"from": "G18", "to": "D5", "strength": 0.45},  
    {"from": "G18", "to": "D11", "strength": 0.35},  
    
    # NF1 associations (G19)  
    {"from": "G19", "to": "D0", "strength": 0.50},  
    {"from": "G19", "to": "D1", "strength": 0.45},  
    {"from": "G19", "to": "D9", "strength": 0.35},  
    {"from": "G19", "to": "D16", "strength": 0.40},  
    {"from": "G19", "to": "D4", "strength": 0.35}  
]  


# Verfication script would confirm all constraints are satisfied
verification_output = verify_gene_disease_association_network(genes, diseases, associations)

print("Gene-Disease Association Network is valid:", verification_output)



Gene-Disease Association Network is valid: (True, {'constraints': {'defined_counts': {'passed': True, 'message': ''}, 'valid_associations': {'passed': True, 'message': ''}, 'degree_constraints': {'passed': True, 'message': ''}, 'duplicate_associations': {'passed': True, 'message': ''}, 'bipartite_constraint': {'passed': True, 'message': ''}}, 'overall_passed': True, 'errors': []})


In [268]:
# import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama32", 
    "GPT4o", "Claude35Sonnet", "Gemini20Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1", "Llama318B", "GrokV3","Claude37Sonnet"]


prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "geneAssociation"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")

verification_json = {
    "genes": genes,
    "diseases": diseases,
    "associations": associations,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


Available Models:
1. GPT4omini
2. Claude35Haiku
3. Gemini20Flash
4. DeepSeekV3
5. Llama32
6. GPT4o
7. Claude35Sonnet
8. Gemini20Pro
9. Llama31405B
10. DeepSeekR1
11. o3mini
12. o1
13. Llama318B
14. GrokV3
15. Claude37Sonnet
Select a model by number: 15

Available Prompt Types:
1. directprompt
2. iterativefeedback
3. programaugmented
Select a prompt type by number: 2

Enter the run number (1-5): 5
Save is succesful! results/geneAssociation_Claude37Sonnet_iterativefeedback_run5_output.json


## Optimal Network Design Challenge

### Problem Description

Design an optimal transportation network represented as a **directed graph** where nodes represent cities and edges represent one-way roads. The network must satisfy the following constraints to ensure efficiency, connectivity, robustness, and cost-effectiveness:

#### 1. Nodes (Cities):
- **Total:** 8, labeled from `C0` to `C7`.
- **Attributes:**
  - **Population:** Number of inhabitants in each city.
    - `C0`: 1,000
    - `C1`: 500
    - `C2`: 750
    - `C3`: 600
    - `C4`: 900
    - `C5`: 400
    - `C6`: 800
    - `C7`: 650

#### 2. Edges (Roads):
- **Definition:** Represents a one-way road from one city to another.
- **Attributes:**
  - **Distance:** Length of the road in kilometers (km). *(Each road must be ≤ 300 km.)*
  - **Construction Cost:** Cost to build the road in thousand dollars ($K).

#### 3. Constraints:
1. **Connectivity:**
   - The network must be **strongly connected**, meaning there is a directed path from any city to every other city.

2. **Road Capacity:**
   - No single road should be longer than **300 km**.

3. **Cost Optimization:**
   - The **total construction cost** of all roads should not exceed **$10,000K**.

4. **Population Accessibility:**
   - Each city must have **at least two incoming roads** to ensure redundancy and accessibility.

5. **Strategic Road Placement:**
   - Cities `C0` and `C7` are major hubs and **must have at least three outgoing roads** each to distribute traffic efficiently.

6. **Avoiding Redundancy:**
   - **No two cities** should have more than **one direct road** connecting them in the same direction.

7. **Minimizing Total Distance:**
   - The **sum of all road distances** should be minimized to ensure efficient transportation.

8. **2-Edge Robustness:**
   - The network must remain strongly connected if **any single road is removed** (i.e. there must be two edge‑disjoint paths between every ordered pair of cities).

9. **Edge-Disjoint Paths Guarantee:**
   - For every pair of distinct cities, there must exist **at least two completely independent (edge‑disjoint) paths** connecting them.

10. **Balanced Outgoing Degree:**
    - Except for the designated hubs (`C0` and `C7`), the difference between the maximum and minimum number of outgoing roads among all cities must not exceed **2**. This prevents “overloaded” junctions.

11. **Path Efficiency Constraint:**
    - For every pair of cities, the shortest route (by total distance) should be less than **500 km** to ensure quick intercity transit.

12. **Cost–Distance Consistency:**
    - For every road, the construction cost (in $K) must be **between 1.0 and 1.5 times its distance (in km)**.  
      *Example:* A road that is 90 km long must have a cost between **90K** and **135K**.
     
13. **Maximum Edge-Hop Constraint:**
    - For every pair of cities, you need to be able to get to every other city in at most 3 edges.

### Required Output Format

Provide the network details in the following Python format. **Strictly adhere to this structure without deviations!**

```python
cities = [
    {"id": "C0", "population": 1000},
    {"id": "C1", "population": 500},
    # ... all cities (C2-C7)
]

roads = [
    {"from": "C0", "to": "C1", "distance": float, "cost": float},
    {"from": "C1", "to": "C2", "distance": float, "cost": float},
    # ... all roads
]


In [326]:
from collections import defaultdict, deque

def verify_optimal_network_design(cities, roads):
    verification_results = {
        "output_structure": {"passed": False, "message": ""},
        "connectivity": {"passed": False, "message": ""},
        "road_capacity": {"passed": False, "message": ""},
        "cost_optimization": {"passed": False, "message": ""},
        "population_accessibility": {"passed": False, "message": ""},
        "strategic_road_placement": {"passed": False, "message": ""},
        "redundancy": {"passed": False, "message": ""},
        "max_edges_constraint": {"passed": False, "message": ""},
        "overall_passed": False,
        "errors": []
    }

    def add_constraint_message(constraint_key, msg):
        """
        Mark the given constraint as failed and append `msg` to its 'message' string,
        as well as appending `msg` to the top-level 'errors' array.
        """
        verification_results[constraint_key]["passed"] = False
        if verification_results[constraint_key]["message"]:
            verification_results[constraint_key]["message"] += " " + msg
        else:
            verification_results[constraint_key]["message"] = msg
        verification_results["errors"].append(msg)

    # 1. Check 'cities' structure
    if not isinstance(cities, list):
        verification_results["output_structure"]["message"] = "'cities' should be a list."
        return False, verification_results
    city_ids = set()
    city_population = {}
    for city in cities:
        if not isinstance(city, dict):
            add_constraint_message("output_structure", "Each city should be a dictionary.")
            continue
        if "id" not in city or "population" not in city:
            cid = city.get("id", "Unknown")
            add_constraint_message("output_structure", f"City {cid} is missing 'id' or 'population'.")
            continue
        cid = city["id"]
        pop = city["population"]
        if cid in city_ids:
            add_constraint_message("output_structure", f"Duplicate city ID detected: {cid}.")
            continue
        city_ids.add(cid)
        city_population[cid] = pop

    # 2. Check 'roads' structure
    if not isinstance(roads, list):
        verification_results["output_structure"]["message"] += " 'roads' should be a list."
        return False, verification_results
    adjacency = defaultdict(list)
    road_set = set()
    total_cost = 0.0
    road_unique = True
    road_capacity_passed = True
    road_capacity_msgs = []
    for road in roads:
        if not isinstance(road, dict):
            add_constraint_message("output_structure", "Each road should be a dictionary.")
            continue
        required_keys = {"from", "to", "distance", "cost"}
        if not required_keys.issubset(road.keys()):
            missing = required_keys - set(road.keys())
            msg = f"Road from {road.get('from', 'Unknown')} to {road.get('to', 'Unknown')} missing {missing}."
            add_constraint_message("output_structure", msg)
            continue
        from_city = road["from"]
        to_city = road["to"]
        dist = road["distance"]
        cost_val = road["cost"]
        if from_city not in city_ids:
            add_constraint_message("output_structure", f"Road 'from' city '{from_city}' is not defined.")
        if to_city not in city_ids:
            add_constraint_message("output_structure", f"Road 'to' city '{to_city}' is not defined.")
        if not isinstance(dist, (int, float)) or dist > 300:
            msg = f"Road from '{from_city}' to '{to_city}' has invalid distance {dist} km (must be <= 300)."
            add_constraint_message("road_capacity", msg)
            road_capacity_passed = False
            road_capacity_msgs.append(msg)
        if not isinstance(cost_val, (int, float)) or cost_val <= 0:
            add_constraint_message("output_structure", f"Road from '{from_city}' to '{to_city}' has invalid cost {cost_val}K (must be positive).")
        road_key = (from_city, to_city)
        if road_key in road_set:
            add_constraint_message("output_structure", f"Duplicate road from '{from_city}' to '{to_city}'.")
            road_unique = False
        else:
            road_set.add(road_key)
        adjacency[from_city].append(to_city)
        total_cost += cost_val

    # Mark output_structure passed if still no error
    if not verification_results["output_structure"]["message"]:
        verification_results["output_structure"]["passed"] = True

    # 3. Road Capacity
    if road_capacity_passed:
        verification_results["road_capacity"]["passed"] = True
    else:
        verification_results["road_capacity"]["passed"] = False
        verification_results["road_capacity"]["message"] = "; ".join(road_capacity_msgs)

    # 4. Cost Optimization
    if total_cost <= 10000:
        verification_results["cost_optimization"]["passed"] = True
    else:
        msg = f"Total construction cost is {total_cost}K, exceeding 10000K limit."
        add_constraint_message("cost_optimization", msg)

    # 5. Connectivity (strongly connected)
    def kosaraju_scc(adj, nodes):
        visited = set()
        stack = []
        def dfs(u):
            visited.add(u)
            for v in adj[u]:
                if v not in visited:
                    dfs(v)
            stack.append(u)
        for u in nodes:
            if u not in visited:
                dfs(u)
        # transpose
        transposed = defaultdict(list)
        for u in adj:
            for v in adj[u]:
                transposed[v].append(u)
        visited.clear()
        sccs = []
        def dfs_t(u, comp):
            visited.add(u)
            comp.append(u)
            for w in transposed[u]:
                if w not in visited:
                    dfs_t(w, comp)
        while stack:
            node = stack.pop()
            if node not in visited:
                comp = []
                dfs_t(node, comp)
                sccs.append(comp)
        return sccs

    scc = kosaraju_scc(adjacency, city_ids)
    if len(scc) == 1:
        verification_results["connectivity"]["passed"] = True
    else:
        msg = "The network is not strongly connected."
        add_constraint_message("connectivity", msg)

    # 6. Population Accessibility (each city needs >=2 incoming roads)
    incoming = defaultdict(int)
    for road in roads:
        incoming[road["to"]] += 1
    pop_access_ok = True
    pop_access_msgs = []
    for cid in city_ids:
        if incoming[cid] < 2:
            pop_access_ok = False
            pop_access_msgs.append(f"City '{cid}' has only {incoming[cid]} incoming roads; needs >=2.")
    if pop_access_ok:
        verification_results["population_accessibility"]["passed"] = True
    else:
        verification_results["population_accessibility"]["passed"] = False
        verification_results["population_accessibility"]["message"] = " ".join(pop_access_msgs)
        verification_results["errors"].extend(pop_access_msgs)

    # 7. Strategic road placement: C0 and C7 need >=3 outgoing roads
    strategic_ok = True
    strategic_msgs = []
    for hub in ["C0", "C7"]:
        out_count = len(adjacency[hub])
        if out_count < 3:
            strategic_ok = False
            strategic_msgs.append(f"Critical city '{hub}' has only {out_count} outgoing roads; needs >=3.")
    if strategic_ok:
        verification_results["strategic_road_placement"]["passed"] = True
    else:
        verification_results["strategic_road_placement"]["passed"] = False
        verification_results["strategic_road_placement"]["message"] = " ".join(strategic_msgs)
        verification_results["errors"].extend(strategic_msgs)

    # 8. Redundancy
    # (We define redundancy as passing connectivity + population_accessibility)
    if verification_results["connectivity"]["passed"] and verification_results["population_accessibility"]["passed"]:
        verification_results["redundancy"]["passed"] = True
    else:
        msg = "Insufficient redundancy due to failing connectivity or population accessibility."
        verification_results["redundancy"]["passed"] = False
        verification_results["redundancy"]["message"] = msg
        verification_results["errors"].append(msg)

    # 9. Output structure is "passed" if road_unique was never set false
    if verification_results["output_structure"]["passed"]:
        # If road_unique is false, mark output_structure as false
        if not road_unique:
            verification_results["output_structure"]["passed"] = False
            if not verification_results["output_structure"]["message"]:
                verification_results["output_structure"]["message"] = "Duplicate roads found."

    # 10. Max edges constraint: BFS up to 3 edges
    max_edges_ok = True
    max_edges_msgs = []
    from collections import deque
    for start in city_ids:
        visited = {start: 0}
        queue = deque([start])
        while queue:
            curr = queue.popleft()
            for nxt in adjacency[curr]:
                if nxt not in visited:
                    visited[nxt] = visited[curr] + 1
                    if visited[nxt] <= 3:
                        queue.append(nxt)
        for cid in city_ids:
            if cid != start:
                if cid not in visited or visited[cid] > 3:
                    max_edges_ok = False
                    max_edges_msgs.append(f"City '{cid}' not reachable from '{start}' in <=3 edges.")
    if max_edges_ok:
        verification_results["max_edges_constraint"]["passed"] = True
    else:
        verification_results["max_edges_constraint"]["passed"] = False
        verification_results["max_edges_constraint"]["message"] = " ".join(max_edges_msgs)
        verification_results["errors"].extend(max_edges_msgs)

    # Overall
    overall = all([
        verification_results["output_structure"]["passed"],
        verification_results["connectivity"]["passed"],
        verification_results["road_capacity"]["passed"],
        verification_results["cost_optimization"]["passed"],
        verification_results["population_accessibility"]["passed"],
        verification_results["strategic_road_placement"]["passed"],
        verification_results["redundancy"]["passed"],
        verification_results["max_edges_constraint"]["passed"]
    ])
    verification_results["overall_passed"] = overall
    return overall, verification_results

cities = [  
    {"id": "C0", "population": 1000},  
    {"id": "C1", "population": 500},  
    {"id": "C2", "population": 750},  
    {"id": "C3", "population": 600},  
    {"id": "C4", "population": 900},  
    {"id": "C5", "population": 400},  
    {"id": "C6", "population": 800},  
    {"id": "C7", "population": 650}  
]  

roads = [  
    {"from": "C0", "to": "C1", "distance": 80.0, "cost": 100.0},  
    {"from": "C1", "to": "C2", "distance": 100.0, "cost": 125.0},  
    {"from": "C2", "to": "C3", "distance": 90.0, "cost": 120.0},  
    {"from": "C3", "to": "C4", "distance": 100.0, "cost": 125.0},  
    {"from": "C4", "to": "C5", "distance": 80.0, "cost": 100.0},  
    {"from": "C5", "to": "C6", "distance": 90.0, "cost": 120.0},  
    {"from": "C6", "to": "C7", "distance": 100.0, "cost": 125.0},  
    {"from": "C7", "to": "C0", "distance": 150.0, "cost": 200.0},  
    {"from": "C0", "to": "C3", "distance": 150.0, "cost": 200.0},  
    {"from": "C0", "to": "C5", "distance": 160.0, "cost": 220.0},  
    {"from": "C7", "to": "C2", "distance": 150.0, "cost": 200.0},  
    {"from": "C7", "to": "C4", "distance": 160.0, "cost": 220.0},  
    {"from": "C2", "to": "C0", "distance": 130.0, "cost": 180.0},  
    {"from": "C4", "to": "C1", "distance": 130.0, "cost": 180.0},  
    {"from": "C6", "to": "C4", "distance": 120.0, "cost": 160.0},  
    {"from": "C3", "to": "C7", "distance": 140.0, "cost": 190.0},  
    {"from": "C5", "to": "C7", "distance": 140.0, "cost": 190.0},  
    {"from": "C1", "to": "C6", "distance": 150.0, "cost": 200.0},  
    {"from": "C6", "to": "C0", "distance": 160.0, "cost": 220.0},  
    {"from": "C1", "to": "C7", "distance": 170.0, "cost": 230.0}  
]  

verification_output = verify_optimal_network_design(cities, roads)
print("Verification Output:", verification_output)


Verification Output: (True, {'output_structure': {'passed': True, 'message': ''}, 'connectivity': {'passed': True, 'message': ''}, 'road_capacity': {'passed': True, 'message': ''}, 'cost_optimization': {'passed': True, 'message': ''}, 'population_accessibility': {'passed': True, 'message': ''}, 'strategic_road_placement': {'passed': True, 'message': ''}, 'redundancy': {'passed': True, 'message': ''}, 'max_edges_constraint': {'passed': True, 'message': ''}, 'overall_passed': True, 'errors': []})


In [328]:
# import os 
import json

models = [
    "GPT4omini", "Claude35Haiku", "Gemini20Flash", "DeepSeekV3", "Llama32", 
    "GPT4o", "Claude35Sonnet", "Gemini20Pro", "Llama31405B", "DeepSeekR1", "o3mini", "o1", "Llama318B", "GrokV3","Claude37Sonnet"]

prompt_types = ["directprompt", "iterativefeedback", "programaugmented"]

problem_type = "cities"


def get_user_input():
    """ Asks the user for model, prompt type, run number, and problem type """
    print("\nAvailable Models:")
    for i, model in enumerate(models):
        print(f"{i+1}. {model}")
    model_choice = int(input("Select a model by number: ")) - 1
    model_selected = models[model_choice]

    print("\nAvailable Prompt Types:")
    for i, prompt in enumerate(prompt_types):
        print(f"{i+1}. {prompt}")
    prompt_choice = int(input("Select a prompt type by number: ")) - 1
    prompt_selected = prompt_types[prompt_choice]

    run_number = int(input("\nEnter the run number (1-5): "))

    return run_number, model_selected, prompt_selected

run_number, model_selected, prompt_selected = get_user_input()

results_dir = "results"
os.makedirs(results_dir, exist_ok=True)

json_file_path = os.path.join(results_dir, f"{problem_type}_{model_selected}_{prompt_selected}_run{run_number}_output.json")


verification_json = {
    "cities":cities,
    "roads":roads,
    "result": verification_output[0],
    "fullOutput": verification_output[1]
}

with open(json_file_path, mode='w', newline='') as json_file:
    json.dump(verification_json, json_file, indent=4)

print("Save is succesful!",json_file_path)


Available Models:
1. GPT4omini
2. Claude35Haiku
3. Gemini20Flash
4. DeepSeekV3
5. Llama32
6. GPT4o
7. Claude35Sonnet
8. Gemini20Pro
9. Llama31405B
10. DeepSeekR1
11. o3mini
12. o1
13. Llama318B
14. GrokV3
15. Claude37Sonnet
Select a model by number: 15

Available Prompt Types:
1. directprompt
2. iterativefeedback
3. programaugmented
Select a prompt type by number: 2

Enter the run number (1-5): 5
Save is succesful! results/cities_Claude37Sonnet_iterativefeedback_run5_output.json


### 